In [1]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm import tqdm

# Config
DATA_DIR = "/Users/ramiab/Desktop/Mineral-Predictions-Local"
GRID_CSV_PATH = os.path.join(DATA_DIR, "Training", "data", "preprocessed", "grid_features.csv")
IMG_DIR = os.path.join(DATA_DIR, "Images", "Grid-Mag-Images")
MODEL_PATH = os.path.join(DATA_DIR, "Training", "models", "cnn_model_20250125_epoch1.pt")
BATCH_SIZE = 64
CHUNK_SIZE = 100000
PRINT_EVERY = 100  # Print every 100 batches
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
LABEL_NAMES = ['AU', 'AG', 'CU', 'CO', 'NI']
OUTPUT_DIR = os.path.join(DATA_DIR, "Training", "data", "output")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Model definition (same as before)
class MineralMultiModalNet(nn.Module):
    def __init__(self, num_classes=5, num_geological_features=None):
        super().__init__()
        
        # CNN Branch
        self.cnn_branch = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        
        # Geological Features Branch
        self.geo_branch = nn.Sequential(
            nn.Linear(num_geological_features, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        
        cnn_output_size = 128 * 21 * 21
        
        # Combination Layer
        self.combiner = nn.Sequential(
            nn.Linear(cnn_output_size + 128, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, img, geo_features):
        x_img = self.cnn_branch(img)
        x_img = x_img.view(x_img.size(0), -1)
        x_geo = self.geo_branch(geo_features)
        combined = torch.cat([x_img, x_geo], dim=1)
        return self.combiner(combined)

class GridPredictionDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.df = pd.read_csv(csv_file)
        self.feature_cols = ['CODE_LITH_encoded', 'STRAT_encoded', 
                           'dist_fault', 'dist_cont']
        self.img_dir = img_dir
        self.transform = transform
        print(f"Grid dataset initialized with {len(self.df)} points")
        print(f"Features being used: {self.feature_cols}")
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_id = int(row['UNIQUE_ID'])
        img_path = os.path.join(self.img_dir, f"{img_id}.jpg")
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        geo_features = torch.tensor(row[self.feature_cols].values, dtype=torch.float32)
        return img, geo_features, img_id

print(f"Loading model from {MODEL_PATH}")
checkpoint = torch.load(MODEL_PATH, map_location=DEVICE)
model = checkpoint['model']
model.eval()
print("Model loaded successfully")

transform = transforms.Compose([
    transforms.Resize((170, 170)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])
])

grid_dataset = GridPredictionDataset(GRID_CSV_PATH, IMG_DIR, transform)
grid_loader = DataLoader(grid_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Initialize storage
all_predictions = []
total_items = len(grid_dataset)
pos_counts = {label: 0 for label in LABEL_NAMES}

print("\nStarting predictions...")
print(f"Total batches: {len(grid_loader)}")
print(f"Will save CSV every {CHUNK_SIZE:,} samples (approximately every {CHUNK_SIZE//BATCH_SIZE:,} batches)")
print(f"Will print status every {PRINT_EVERY} batches")

with torch.no_grad():
    for batch_idx, (images, geo_features, uids) in enumerate(tqdm(grid_loader, desc="Predicting")):
        images = images.to(DEVICE)
        geo_features = geo_features.to(DEVICE)
        
        outputs = model(images, geo_features)
        probs = torch.sigmoid(outputs).cpu().numpy()
        preds = (probs >= 0.5).astype(int)
        uids = uids.numpy()
        
        batch_pos_counts = {label: 0 for label in LABEL_NAMES}
        
        for i in range(len(uids)):
            row_dict = {'UNIQUE_ID': uids[i]}
            for j, label in enumerate(LABEL_NAMES):
                row_dict[f'{label}_prob'] = probs[i][j]
                row_dict[f'{label}_pred'] = preds[i][j]
                if preds[i][j] == 1:
                    pos_counts[label] += 1
                    batch_pos_counts[label] += 1
            all_predictions.append(row_dict)
        
        # Print mini-status every 100 batches
        if (batch_idx + 1) % PRINT_EVERY == 0:
            samples_processed = (batch_idx + 1) * BATCH_SIZE
            print(f"\nBatch {batch_idx + 1}/{len(grid_loader)} ({samples_processed:,} samples)")
            print("Batch predictions:")
            for label in LABEL_NAMES:
                print(f"{label}: {batch_pos_counts[label]} positives in this batch")
        
        # Save chunk if needed
        processed_so_far = (batch_idx + 1) * BATCH_SIZE
        if processed_so_far % CHUNK_SIZE < BATCH_SIZE or processed_so_far >= total_items:
            if processed_so_far > total_items:
                processed_so_far = total_items
                
            predictions_df = pd.DataFrame(all_predictions)
            original_df = pd.read_csv(GRID_CSV_PATH)
            original_df['UNIQUE_ID'] = original_df['UNIQUE_ID'].astype(int)
            merged_df = pd.merge(original_df, predictions_df, on='UNIQUE_ID', how='inner')
            
            out_name = f"CNN_output_batch_{processed_so_far}.csv"
            out_path = os.path.join(OUTPUT_DIR, out_name)
            merged_df.to_csv(out_path, index=False)
            
            print(f"\n{'='*50}")
            print(f"SAVED CHUNK: {out_path}")
            print(f"Processed {processed_so_far:,}/{total_items:,} points ({processed_so_far/total_items*100:.1f}%)")
            print("\nCumulative prediction statistics:")
            for label in LABEL_NAMES:
                pos_pct = (pos_counts[label] / processed_so_far) * 100
                print(f"{label}: {pos_counts[label]:,} positive predictions ({pos_pct:.2f}%)")
            print(f"{'='*50}\n")

print("\nPrediction complete!")
print("Final statistics:")
for label in LABEL_NAMES:
    pos_pct = (pos_counts[label] / total_items) * 100
    print(f"{label}: {pos_counts[label]:,} total positive predictions ({pos_pct:.2f}%)")

Loading model from /Users/ramiab/Desktop/Mineral-Predictions-Local/Training/models/cnn_model_20250125_epoch1.pt


/var/folders/3x/4p2fydpn6_3cl5dk1hyl2ppw0000gn/T/ipykernel_11836/1545853793.py:104: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(MODEL_PATH, map_loc

Model loaded successfully
Grid dataset initialized with 1107289 points
Features being used: ['CODE_LITH_encoded', 'STRAT_encoded', 'dist_fault', 'dist_cont']

Starting predictions...
Total batches: 17302
Will save CSV every 100,000 samples (approximately every 1,562 batches)
Will print status every 100 batches


Predicting:   1%|▏                        | 100/17302 [02:04<5:13:05,  1.09s/it]


Batch 100/17302 (6,400 samples)
Batch predictions:
AU: 0 positives in this batch
AG: 3 positives in this batch
CU: 0 positives in this batch
CO: 0 positives in this batch
NI: 0 positives in this batch


...


Batch 17200/17302 (1,100,800 samples)
Batch predictions:
AU: 0 positives in this batch
AG: 7 positives in this batch
CU: 0 positives in this batch
CO: 0 positives in this batch
NI: 4 positives in this batch


Predicting: 100%|██████████████████████▉| 17300/17302 [3:40:30<00:01,  1.88it/s]


Batch 17300/17302 (1,107,200 samples)
Batch predictions:
AU: 0 positives in this batch
AG: 4 positives in this batch
CU: 0 positives in this batch
CO: 0 positives in this batch
NI: 0 positives in this batch


Predicting: 100%|███████████████████████| 17302/17302 [3:40:39<00:00,  1.31it/s]


SAVED CHUNK: /Users/ramiab/Desktop/Mineral-Predictions-Local/Training/data/output/CNN_output_batch_1107289.csv
Processed 1,107,289/1,107,289 points (100.0%)

Cumulative prediction statistics:
AU: 12,202 positive predictions (1.10%)
AG: 70,201 positive predictions (6.34%)
CU: 17,552 positive predictions (1.59%)
CO: 31,365 positive predictions (2.83%)
NI: 52,032 positive predictions (4.70%)


Prediction complete!
Final statistics:
AU: 12,202 total positive predictions (1.10%)
AG: 70,201 total positive predictions (6.34%)
CU: 17,552 total positive predictions (1.59%)
CO: 31,365 total positive predictions (2.83%)
NI: 52,032 total positive predictions (4.70%)
